## Bank PDF Query Using Langchain

In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [11]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI Secret here"

In [4]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('bank.pdf')

In [5]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
raw_text

'Loans Description  \nSmart bank mainl y has 5 type s of loan schemes.  \n1. Overdra ft Facilities  \n2. Housing Loans  \n3. Education Loans  \n4. Pensioners ’ Loans  \n5. Personal Loans  \n1. Over draft  Facilities  \nAn overdraft is the ideal way to manage your cash flow at competitive rates of interest at speed. It is \nlinked to your current account and you can use as much as you need, up to your limit. You can also apply \nfor and either increase or decrease your limit through Internet banking. The controls are in your hands.  \n• An overdraft is quick and easy to arrange  \n• The cash is available when you need it  \n• You only pay interest on what you use, not on the full amount of your set limit  \n• Sole proprietors can apply for an overdraft or increase their limits   \n2. Housing Loans  \n2.1. Housing Loan Sche me \n• Purchase a land to construct a house later  \n• Purchase a land and to construct a house  \n• Construct a house in a land owned by the borrower/s  \n• Purchase

In [13]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 400,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [14]:
len(texts)

58

In [15]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [16]:
document_search = FAISS.from_texts(texts, embeddings)

In [17]:
document_search


In [18]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [19]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [20]:
query = "what is the proceedure for overdrafts"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The procedure for overdrafts is quick and easy to arrange. You can apply for an overdraft or increase your limit through internet banking, and the controls are in your hands. The cash is available when you need it, and you only pay interest on what you use, not on the full amount of your set limit. Sole proprietors can also apply for an overdraft or increase their limits.'

In [21]:
query = "What is the Quantum of the Loan"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The Quantum of the Loan for this specific housing loan scheme is a maximum of Rs. 50 Million, with a repayment period of 25 years and available for all island residents.'

In [22]:
query = "what is the max scheme of the above"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The maximum loan amount for this housing loan scheme is Rs. 10.0 Million, or 80% of the course fee, whichever is less.'